<a href="https://colab.research.google.com/github/nalrob/Datos_Masivos_MCD/blob/main/Practica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 1: Paralelizacion.

## Parte 1: Investigar sobre las lazy variables y como se usa el Delay para problemas mas grandes.


"Lazy Variable" o variable perezosa es una variable que se inicializa solo cuando se llama por primera vez.
Esta técnica se utiliza para mejorar el rendimiento y la eficiencia de los programas, ya que puede ahorrar tiempo y recursos al no calcular o inicializar valores que nunca se utilizarán.


Ejemplo de su uso:

In [ ]:
from functools import lru_cache

class Person:
    def __init__(self, weight, height):
        self.weight = weight
        self.height = height

    @property
    @lru_cache(maxsize=None)
    def BMIIndex(self):
        print("Calculando el índice de masa corporal...")
        return self.weight / pow(self.height, 2)

# Crear una instancia de la clase Person
person = Person(70.0, 1.75)

# Acceder a la propiedad perezosa BMIIndex
print(person.BMIIndex)

Calculando el índice de masa corporal...
22.857142857142858



En programación, existen diferentes tipos de variables "lazy" o perezosas, que se utilizan en diferentes contextos y para diferentes propósitos. Algunos de los tipos más comunes de variables "lazy" son:

**1.   Lazy evaluation**
 
> Se utiliza en lenguajes de programación que tienen una evaluación perezosa, donde las expresiones no se evalúan hasta que se utilizan. Esto significa que los valores solo se calculan cuando es necesario, lo que puede mejorar el rendimiento y la eficiencia del programa.



**2.   Lazy initialization**


> Se utiliza para retrasar la inicialización de una variable hasta que sea necesaria. Por ejemplo, una variable que se utiliza raramente puede inicializarse perezosamente para evitar la sobrecarga de la inicialización al inicio del programa.




**3.    Lazy loading**


> Se utiliza para cargar datos o recursos solo cuando son necesarios. Por ejemplo, en una aplicación web, se pueden cargar imágenes o archivos solo cuando se muestran en la página web, lo que mejora el rendimiento y la 


**4.   Memorization**

> Se utiliza para almacenar el resultado de una función costosa en caché y devolver el resultado almacenado en lugar de volver a calcularlo cuando se llama a la función con los mismos parámetros. Esto mejora significativamente el rendimiento de la función en llamadas repetidas con los mismos parámetros.


**5.   Thunk**


> Es una función que se utiliza para retrasar la evaluación de una expresión hasta que sea necesaria. El término "thunk" proviene de la combinación de "think" y "chunk", ya que la función se utiliza para pensar en una porción de código y evaluarla solo cuando sea necesario.



## Parte 2: Generen varias funciones y construyan un grafo de paralelización con al menos 4 cuellos de botella.



El código genera un ticket de compra en el que se procesa el registro de venta.

In [ ]:
import dask.delayed as delayed
from dask.diagnostics import ProgressBar
import time
import numpy as np
import random

In [ ]:
class OrderLine():
  taxes = 0
  total = 0
  subtotal = 0
  type = 1

  def __init__(self):
    self.subtotal = random.randint(100, 5000)
    self.type =random.randint(1,3)

  def calculate_taxes(self):
    tax = 0.32 if self.type == 1 else 0.16
    self.taxes = self.subtotal * tax

  def round_taxes(self):
    self.taxes = round(self.taxes, 2)

  def calculate_total(self):
    self.total = self.subtotal + self.taxes

In [ ]:
orders = []
for n in range(1000):
  orders.append(OrderLine())

1 Cuello de botella: Calcular Sumatoria de subtotal


In [ ]:
for order in orders:
  order.round_taxes()

In [ ]:


#CB 1
def calculate_taxes(x):
    tax = 0.32 if type == 1 else 0.16
    taxes = x * tax
    return taxes


#CB 2
def round_taxes(x):
    r_taxes= round(taxes, 2) 
    return r_taxes

#CB 3 

def calculate_total(self):
    total = r_taxes + data
  

In [ ]:
step1 = [delayed(calculate_taxes)(i) for i in data]
step2 = [delayed(round_taxes)(k) for k in data]
total = delayed(sum)(step2)


In [ ]:
step1 = [delayed(calculate_taxes)(i) for i in data]

2 Cuello de botella: Calcular Sumatoria de taxes


In [ ]:
for order in orders:
  order.round_taxes()

3 Cuello de botella: Calcular Sumatoria de taxes redondeados


In [ ]:
for order in orders:
  order.calculate_total()

4 Cuello de botella: Calcular Sumatoria de total

In [ ]:
orders[100].subtotal

3739


Extra challenge: Buscar ensemble learning en Scikit learn, utilizar minimo 3 algoritmos de machine learning para crear un modelo paralelo basado en un ensamble. Generar uno con dask y otro con la libreria de ensemble de sklearn, y comparar resultados. 